https://adventofcode.com/2020/day/7

In [47]:
from collections import deque

import networkx as nx
from toolz import partition

In [48]:
datafile = 'data/07-1.txt'

In [49]:
def parse_line(s):
    """
    'pale indigo bags contain 3 pale orange bags, 5 muted bronze bags, 2 pale tan bags, 3 posh fuchsia bags.'
    ->
    [('pale indigo', 'pale orange', 3),
     ('pale indigo', 'muted bronze', 5),
     ('pale indigo', 'pale tan', 2),
     ('pale indigo', 'posh fuchsia', 3)]
    """
    tokens = s.split()
    if tokens[4] == 'no':
        return []
    outer = ' '.join(tokens[:2])
    bags = []
    for subtokens in partition(4, tokens[4:]):
        count = int(subtokens[0])
        inner = ' '.join(subtokens[1:3])
        bags.append((outer, inner, count))
    return bags

In [50]:
line = 'pale indigo bags contain 3 pale orange bags, 5 muted bronze bags, 2 pale tan bags, 3 posh fuchsia bags.'
parse_line(line)

[('pale indigo', 'pale orange', 3),
 ('pale indigo', 'muted bronze', 5),
 ('pale indigo', 'pale tan', 2),
 ('pale indigo', 'posh fuchsia', 3)]

In [51]:
def parse_data(lines):
    data = []
    for line in lines:
        data.extend(parse_line(line))
    return data

In [52]:
with open(datafile) as fh:
    data = parse_data(fh)

In [53]:
len(data), data[:10]

(1403,
 [('wavy green', 'posh black', 1),
  ('wavy green', 'faded green', 1),
  ('wavy green', 'wavy red', 4),
  ('dotted chartreuse', 'light beige', 1),
  ('dark white', 'dotted white', 2),
  ('clear aqua', 'posh orange', 4),
  ('clear aqua', 'pale blue', 4),
  ('faded green', 'plaid orange', 2),
  ('faded green', 'drab green', 4),
  ('faded green', 'pale aqua', 4)])

In [8]:
data[-3:]

[('clear silver', 'drab turquoise', 3),
 ('clear silver', 'drab purple', 4),
 ('vibrant cyan', 'vibrant plum', 5)]

In [9]:
DG = nx.DiGraph()
DG.add_weighted_edges_from(data)

In [10]:
DG['shiny gold']

AtlasView({'dotted magenta': {'weight': 3}, 'shiny beige': {'weight': 2}, 'plaid brown': {'weight': 3}, 'clear indigo': {'weight': 5}})

In [11]:
def count_outers(bag, g=DG):
    tocheck = set(g.predecessors(bag))
    preds = set()
    while tocheck:
        node = tocheck.pop()
        preds.add(node)
        tocheck.update(set(g.predecessors(node)).difference(preds))
    return len(preds)

# 1

In [12]:
count_outers('shiny gold')

169

In [13]:
node = DG['shiny gold']
node

AtlasView({'dotted magenta': {'weight': 3}, 'shiny beige': {'weight': 2}, 'plaid brown': {'weight': 3}, 'clear indigo': {'weight': 5}})

In [14]:
list(node.items())

[('dotted magenta', {'weight': 3}),
 ('shiny beige', {'weight': 2}),
 ('plaid brown', {'weight': 3}),
 ('clear indigo', {'weight': 5})]

In [15]:
def count_inners(bag, g=DG):
    count = 0
    q = deque()
    for node, attrs in g[bag].items():
        n = attrs['weight']
        q.extend([node] * n)
        count += n
    while q:
        bag = q.popleft()
        for node, attrs in g[bag].items():
            n = attrs['weight']
            q.extend([node] * n)
            count += n
    return count

# 2

In [16]:
count_inners('shiny gold')

82372

## reddit

In [39]:
from collections import deque

import networkx as nx
from toolz import partition_all

def parse_data(lines):
    data = []
    for line in lines:
        data.extend(parse_line(line))
    return data

def parse_line(s):
    """
    'pale indigo bags contain 3 pale orange bags, 5 muted bronze bags, 2 pale tan bags, 3 posh fuchsia bags.'
    ->
    [('pale indigo', 'pale orange', 3),
     ('pale indigo', 'muted bronze', 5),
     ('pale indigo', 'pale tan', 2),
     ('pale indigo', 'posh fuchsia', 3)]
    """
    tokens = s.split()
    if tokens[4] == 'no':
        return []
    outer = ' '.join(tokens[:2])
    bags = []
    for subtokens in partition_all(4, tokens[4:]):
        count = int(subtokens[0])
        inner = ' '.join(subtokens[1:3])
        bags.append((outer, inner, count))
    return bags

with open(datafile) as fh:
    data = parse_data(fh)

DG = nx.DiGraph()
DG.add_weighted_edges_from(data)

def count_outers(bag, g=DG):
    tocheck = set(g.predecessors(bag))
    preds = set()
    while tocheck:
        node = tocheck.pop()
        preds.add(node)
        tocheck.update(set(g.predecessors(node)).difference(preds))
    return len(preds)

part_1 = count_outers('shiny gold')

def count_inners(bag, g=DG):
    count = 0
    q = deque()
    
    def count_bag(b):
        nonlocal count
        for node, attrs in g[b].items():
            n = attrs['weight']
            q.extend([node] * n)
            count += n

    count_bag(bag)
    while q:
        count_bag(q.popleft())
    return count

part_2 = count_inners('shiny gold')

In [40]:
part_1, part_2

(169, 82372)

In [45]:
len(nx.ancestors(DG, 'shiny gold'))

169

## improved

In [58]:
import networkx as nx
from toolz import partition

def parse_data(lines):
    data = []
    for line in lines:
        data.extend(parse_line(line))
    return data

def parse_line(s):
    """
    'pale indigo bags contain 3 pale orange bags, 5 muted bronze bags, 2 pale tan bags, 3 posh fuchsia bags.'
    ->
    [('pale indigo', 'pale orange', 3),
     ('pale indigo', 'muted bronze', 5),
     ('pale indigo', 'pale tan', 2),
     ('pale indigo', 'posh fuchsia', 3)]
    """
    tokens = s.split()
    if tokens[4] == 'no':
        return []
    outer = ' '.join(tokens[:2])
    bags = []
    for subtokens in partition(4, tokens[4:]):
        count = int(subtokens[0])
        inner = ' '.join(subtokens[1:3])
        bags.append((outer, inner, count))
    return bags

with open(datafile) as fh:
    data = parse_data(fh)

DG = nx.DiGraph()
DG.add_weighted_edges_from(data)

part_1 = len(nx.ancestors(DG, 'shiny gold'))

def count_inners(bag, g=DG):
    count = 0
    stack = []
    
    def count_bag(b):
        nonlocal count
        for node, attrs in g[b].items():
            n = attrs['weight']
            stack.extend([node] * n)
            count += n

    count_bag(bag)
    while stack:
        count_bag(stack.pop())
    return count

part_2 = count_inners('shiny gold')

print(part_1)
print(part_2)

169
82372
